# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

# Approach

Approach for Question 1-4:

At first we will process the data related to the metropoliton regions and extract the corresponding **team names** for a particular sport. Then we will calculate the **win/loss ratio** for each team of the corresponding sport and assign them to the first dataframe with metropoliton data.

Approach for question 5:

Most processing needed for this problem are completed in Questions 1-4. Sadly, the functions in Q1-Q4 don't return the dataframe. So, we have to do all the processing done in Q1-Q4 in Q5 also. Then we will calculate **t-test score** between **total win/loss ratio** for each sport teams in a city.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html", na_values=['—', '— [note 22]'])[1]
cities=cities.iloc[:-1,[0,3,4,5,6,7,8]] #keep only important columns
cities = cities.replace('^\[note\s\d*\]', np.nan, regex=True) #replace [note ..] with np.nan
cities = cities.replace('\[note\s\d*\]', '', regex=True) #replace succeeding the [note ..]


def nhl_correlation():     
    #-----------------------Process Metropoliton cities data----------------------------
    # for this we need only NHL data. So, drop the cities that don't have any NHL teams
    cities_nhl = cities.dropna(subset=['NHL']) #drop cities w/o NHL teams
    cols_cities_nhl = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_nhl = cities_nhl.rename(columns=cols_cities_nhl)
    cities_nhl = cities_nhl[['metro', 'pop', 'NHL']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_nhl['pop'] = cities_nhl['pop'].astype('int')
    cities_nhl = cities_nhl.reset_index(drop=True) #reset_index
    
    def split_and_keep(words):
        """
        This function divides team names connected together and returns a series
        (return value is series instead of list to make it easier to convert into dataframe)
        """
        lst = re.split('([a-z][A-Z0-9])',words)
        new_lst = lst[:]
        length = len(lst)
        for i, item in enumerate(lst):
            if i%2 != 0 and i != length-1:
                temp = list(item)
                new_lst[i-1] = new_lst[i-1]+temp[0]
                new_lst[i+1] = temp[1]+new_lst[i+1]
        final_lst = [new_item for j, new_item in enumerate(new_lst) if j%2 == 0]
        return pd.Series(final_lst)
    
    cities_nhl[['nhl1', 'nhl2', 'nhl3']] = pd.DataFrame(cities_nhl['NHL'].apply(split_and_keep))
    
    #----------------------process NHL data----------------------------
    nhl_df_18 = nhl_df[nhl_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples before dropping:{nhl_df_18.shape[0]}") #(3*8+1*7+4)
    # now keep only the columns we need
    cols_nhl = ['team', 'W', 'L', 'OL']
    nhl_df_18 = nhl_df_18[cols_nhl]
    # now drop the rows containing information about division
    nhl_df_18 = nhl_df_18.replace('\sDivision$', np.nan, regex=True).dropna().reset_index(drop=True)
    # drop the asterisk(*) from team names
    nhl_df_18['team'] = nhl_df_18['team'].replace('\*$', '', regex=True)
    # the W, L, OL columns contain string. So convert those to integer
    nhl_df_18[['W', 'L', 'OL']] = nhl_df_18[['W', 'L', 'OL']].astype('int')
    # add a new row containing the intended ratio
#     nhl_df_18['ratio'] = nhl_df_18['W'] / (nhl_df_18['W']+nhl_df_18['L']+nhl_df_18['OL'])
    nhl_df_18['ratio'] = nhl_df_18['W'] / (nhl_df_18['L']+nhl_df_18['W'])
    # print(f"no of data samples before dropping:{nhl_df_18.shape[0]}") #(3*8+1*7)
    # remove space from city part of team names
    nhl_df_18['team'] = nhl_df_18['team'].replace({'Tampa Bay': 'TampaBay', 'New Jersey':'NewJersey', 'Los Angeles': 'LosAngeles',
                                                   'St. Louis':'St.Louis', 'San Jose':'SanJose', 'New York':'NewYork'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    nhl_df_18['team'] = nhl_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))
    
    #------------------------- Merging---------------------------------
    merged_nhl1 = cities_nhl.merge(nhl_df_18[['team', 'ratio']], how='left', left_on='nhl1', right_on='team')
    merged_nhl2 = merged_nhl1.merge(nhl_df_18[['team', 'ratio']], how='left', left_on='nhl2', right_on='team')
    merged_nhl3 = merged_nhl2.merge(nhl_df_18[['team', 'ratio']], how='left', left_on='nhl3', right_on='team')
    
    # add the total win/loss ratio to the cities dataframe
    divide_value = 1*merged_nhl3['ratio_x'].notna() + 1*merged_nhl3['ratio_y'].notna() + 1*merged_nhl3['ratio'].notna()
    cities_nhl['total_ratio'] = (merged_nhl3['ratio_x'] + merged_nhl3['ratio_y'].fillna(0) + merged_nhl3['ratio'].fillna(0))/divide_value
    
#     raise NotImplementedError()
    
    population_by_region = cities_nhl['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = cities_nhl['total_ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html", na_values=['—', '— [note 22]'])[1]
    cities=cities.iloc[:-1,[0,3,4,5,6,7,8]] #keep only important columns
    cities = cities.replace('^\[note\s\d*\]', np.nan, regex=True) #replace [note ..] with np.nan
    cities = cities.replace('\[note\s\d*\]', '', regex=True) #replace succeeding the [note ..]
    
    #---------------------processing cities data----------------------
    # for this we need only NBA data. So, drop the cities that don't have any NBA teams
    cities_nba = cities.dropna(subset=['NBA']) #drop cities w/o NBA teams
    cols_cities_nba = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_nba = cities_nba.rename(columns=cols_cities_nba)
    cities_nba = cities_nba[['metro', 'pop', 'NBA']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_nba['pop'] = cities_nba['pop'].astype('int')
    cities_nba = cities_nba.reset_index(drop=True) #reset_index
    
    def split_and_keep(words):
        """
        This function divides team names connected together and returns a series
        (return value is series instead of list to make it easier to convert into dataframe)
        """
        lst = re.split('([a-z][A-Z0-9])',words)
        new_lst = lst[:]
        length = len(lst)
        for i, item in enumerate(lst):
            if i%2 != 0 and i != length-1:
                temp = list(item)
                new_lst[i-1] = new_lst[i-1]+temp[0]
                new_lst[i+1] = temp[1]+new_lst[i+1]
        final_lst = [new_item for j, new_item in enumerate(new_lst) if j%2 == 0]
        return pd.Series(final_lst)
    
    cities_nba[['nba1', 'nba2']] = pd.DataFrame(cities_nba['NBA'].apply(split_and_keep))
    
    #--------------------Process NBA data------------------------
    nba_df_18 = nba_df[nba_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples:{nba_df_18.shape[0]}") 
    # now keep only the columns we need
    cols_nba = ['team', 'W', 'L']
    nba_df_18 = nba_df_18[cols_nba]
    # drop the asterisk(*) from team names
    nba_df_18['team'] = nba_df_18['team'].replace('\*\s\(.\)$|\s\(.*\)$', '', regex=True)
    # the W, L columns contain string. So convert those to integer
    nba_df_18[['W', 'L']] = nba_df_18[['W', 'L']].astype('int')
    # add a new row containing the intended ratio
    nba_df_18['ratio'] = nba_df_18['W'] / (nba_df_18['W']+nba_df_18['L'])
    # remove space from city part of team names
    nba_df_18['team'] = nba_df_18['team'].replace({'Los Angeles': 'LosAngeles', 'New York':'NewYork', 
                                                   'Golden State':'GoldenState', 'Oklahoma City':'OklahomaCity', 
                                                   'New Orleans':'NewOrleans', 'San Antonio':'SanAntonio'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    nba_df_18['team'] = nba_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))
    
    #-----------------Merge dataframe-------------------
    merged_nba1 = cities_nba.merge(nba_df_18[['team', 'ratio']], how='left', left_on='nba1', right_on='team')
    merged_nba2 = merged_nba1.merge(nba_df_18[['team', 'ratio']], how='left', left_on='nba2', right_on='team')
    
    # add the total win/loss ratio to the cities dataframe
    divide_value_nba = 1*merged_nba2['ratio_x'].notna() + 1*merged_nba2['ratio_y'].notna() 
    cities_nba['total_ratio'] = (merged_nba2['ratio_x'] + merged_nba2['ratio_y'].fillna(0))/divide_value_nba

    # raise NotImplementedError()
    
    population_by_region = cities_nba['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = cities_nba['total_ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    
    #------------read data--------------------
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html", na_values=['—', '— [note 22]'])[1]
    cities=cities.iloc[:-1,[0,3,4,5,6,7,8]] #keep only important columns
    cities = cities.replace('^\[note\s\d*\]', np.nan, regex=True) #replace [note ..] with np.nan
    cities = cities.replace('\[note\s\d*\]', '', regex=True) #replace succeeding the [note ..]
    
    #-------------Process cities data----------------
    # for this we need only MLB data. So, drop the cities that don't have any MLB teams
    cities_mlb = cities.dropna(subset=['MLB']) #drop cities w/o MLB teams
    cols_cities_mlb = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_mlb = cities_mlb.rename(columns=cols_cities_mlb)
    cities_mlb = cities_mlb[['metro', 'pop', 'MLB']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_mlb['pop'] = cities_mlb['pop'].astype('int')
    cities_mlb = cities_mlb.reset_index(drop=True) #reset_index
    
    def split_and_keep(words):
        """
        This function divides team names connected together and returns a series
        (return value is series instead of list to make it easier to convert into dataframe)
        """
        lst = re.split('([a-z][A-Z0-9])',words)
        new_lst = lst[:]
        length = len(lst)
        for i, item in enumerate(lst):
            if i%2 != 0 and i != length-1:
                temp = list(item)
                new_lst[i-1] = new_lst[i-1]+temp[0]
                new_lst[i+1] = temp[1]+new_lst[i+1]
        final_lst = [new_item for j, new_item in enumerate(new_lst) if j%2 == 0]
        return pd.Series(final_lst)
    cities_mlb[['mlb1', 'mlb2']] = pd.DataFrame(cities_mlb['MLB'].apply(split_and_keep))
    
    #--------------------Process MLB data----------------------
    mlb_df_18 = mlb_df[mlb_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples:{mlb_df_18.shape[0]}") 
    # now keep only the columns we need
    cols_mlb = ['team', 'W', 'L']
    mlb_df_18 = mlb_df_18[cols_mlb]
    # the W, L columns contain integer already. however make sure they are integer
    mlb_df_18[['W', 'L']] = mlb_df_18[['W', 'L']].astype('int')
    # add a new row containing the intended ratio
    mlb_df_18['ratio'] = mlb_df_18['W'] / (mlb_df_18['W']+mlb_df_18['L'])
    # remove space from city part of team names
    mlb_df_18['team'] = mlb_df_18['team'].replace({'Los Angeles': 'LosAngeles', 'New York':'NewYork', 'St. Louis':'StLouis', 
                                                   'Tampa Bay':'TampaBay', 'Kansas City':'KansasCity', 
                                                   'San Francisco':'SanFrancisco', 'San Diego':'SanDiego'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    mlb_df_18['team'] = mlb_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))
    
    #-------------Merge Data-----------------------
    merged_mlb1 = cities_mlb.merge(mlb_df_18[['team', 'ratio']], how='left', left_on='mlb1', right_on='team')
    merged_mlb2 = merged_mlb1.merge(mlb_df_18[['team', 'ratio']], how='left', left_on='mlb2', right_on='team')
    
    # add the total win/loss ratio to the cities dataframe
    divide_value_mlb = 1*merged_mlb2['ratio_x'].notna() + 1*merged_mlb2['ratio_y'].notna() 
    cities_mlb['total_ratio'] = (merged_mlb2['ratio_x'] + merged_mlb2['ratio_y'].fillna(0))/divide_value_mlb

    
#   raise NotImplementedError()
    
    population_by_region = cities_mlb['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = cities_mlb['total_ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    
    #--------------------Reading and some preprocessing of cities data---------------------
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html", na_values=['—', '— [note 22]'])[1]
    cities=cities.iloc[:-1,[0,3,4,5,6,7,8]] #keep only important columns
    cities = cities.replace('^\[note\s\d*\]', np.nan, regex=True) #replace [note ..] with np.nan
    cities = cities.replace('\[note\s\d*\]', '', regex=True) #replace succeeding the [note ..]
    
    #--------------------More preprocessing of cities data-----------------------------
    # for this we need only NFL data. So, drop the cities that don't have any NFL teams
    cities_nfl = cities.dropna(subset=['NFL']) #drop cities w/o NFL teams
    cols_cities_nfl = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_nfl = cities_nfl.rename(columns=cols_cities_nfl)
    cities_nfl = cities_nfl[['metro', 'pop', 'NFL']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_nfl['pop'] = cities_nfl['pop'].astype('int')
    cities_nfl = cities_nfl.reset_index(drop=True) #reset_index

    def split_and_keep(words):
        """
        This function divides team names connected together and returns a series
        (return value is series instead of list to make it easier to convert into dataframe)
        """
        lst = re.split('([a-z][A-Z0-9])',words)
        new_lst = lst[:]
        length = len(lst)
        for i, item in enumerate(lst):
            if i%2 != 0 and i != length-1:
                temp = list(item)
                new_lst[i-1] = new_lst[i-1]+temp[0]
                new_lst[i+1] = temp[1]+new_lst[i+1]
        final_lst = [new_item for j, new_item in enumerate(new_lst) if j%2 == 0]
        return pd.Series(final_lst)
    cities_nfl[['nfl1', 'nfl2']] = pd.DataFrame(cities_nfl['NFL'].apply(split_and_keep))
    
    #----------------Preprocessing NFL data----------------------
    nfl_df_18 = nfl_df[nfl_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples:{nfl_df_18.shape[0]}") 
    # now keep only the columns we need
    cols_nfl = ['team', 'W', 'L']
    nfl_df_18 = nfl_df_18[cols_nfl]
    # now drop the rows containing information about division
    nfl_df_18 = nfl_df_18.replace('[AN]FC\s', np.nan, regex=True).dropna().reset_index(drop=True)
    # print(f"no of data samples after removing division row:{nfl_df_18.shape[0]}")
    # drop the asterisk(*) or plus(+) from team names
    nfl_df_18['team'] = nfl_df_18['team'].replace('[\*\+]', '', regex=True)
    # the W, L columns contain string, so convert them to integer
    nfl_df_18[['W', 'L']] = nfl_df_18[['W', 'L']].astype('int')
    # add a new row containing the intended ratio
    nfl_df_18['ratio'] = nfl_df_18['W'] / (nfl_df_18['W']+nfl_df_18['L'])
    # # remove space from city part of team names
    nfl_df_18['team'] = nfl_df_18['team'].replace({'New England':'NewEngland','New York':'NewYork','Kansas City':'KansasCity',
                                                   'Los Angeles': 'LosAngeles', 'Green Bay':'GreenBay', 'St. Louis':'StLouis',
                                                   'New Orleans':'NewOrleans','Tampa Bay':'TampaBay',  
                                                   'San Francisco':'SanFrancisco'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    nfl_df_18['team'] = nfl_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))
    
    #---------------------Merging---------------------------
    merged_nfl1 = cities_nfl.merge(nfl_df_18[['team', 'ratio']], how='left', left_on='nfl1', right_on='team')
    merged_nfl2 = merged_nfl1.merge(nfl_df_18[['team', 'ratio']], how='left', left_on='nfl2', right_on='team')
    # add the total win/loss ratio to the cities dataframe
    divide_value_nfl = 1*merged_nfl2['ratio_x'].notna() + 1*merged_nfl2['ratio_y'].notna() 
    cities_nfl['total_ratio'] = (merged_nfl2['ratio_x'] + merged_nfl2['ratio_y'].fillna(0))/divide_value_nfl
    
#     raise NotImplementedError()
    
    population_by_region = cities_nfl['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = cities_nfl['total_ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    
    #-----------------------Read and preprocess cities data--------------------------
    cities=pd.read_html("assets/wikipedia_data.html", na_values=['—', '— [note 22]'])[1]
    cities=cities.iloc[:-1,[0,3,4,5,6,7,8]] #keep only important columns
    cities = cities.replace('^\[note\s\d*\]', np.nan, regex=True) #replace [note ..] with np.nan
    cities = cities.replace('\[note\s\d*\]', '', regex=True) #replace succeeding the [note ..]
    cols_cities = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'}
    cities = cities.rename(columns=cols_cities)
    
    #-----------------------------------------------------------------------------------
    #--------------------------------------NFL Data-------------------------------------
    #-----------------------------------------------------------------------------------
    # for this we need only NFL data. So, drop the cities that don't have any NFL teams
    cities_nfl = cities.dropna(subset=['NFL']) #drop cities w/o NFL teams
    cols_cities_nfl = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_nfl = cities_nfl.rename(columns=cols_cities_nfl)
    cities_nfl = cities_nfl[['metro', 'pop', 'NFL']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_nfl['pop'] = cities_nfl['pop'].astype('int')
    cities_nfl = cities_nfl.reset_index(drop=True) #reset_index

    def split_and_keep(words):
        """
        This function divides team names connected together and returns a series
        (return value is series instead of list to make it easier to convert into dataframe)
        """
        lst = re.split('([a-z][A-Z0-9])',words)
        new_lst = lst[:]
        length = len(lst)
        for i, item in enumerate(lst):
            if i%2 != 0 and i != length-1:
                temp = list(item)
                new_lst[i-1] = new_lst[i-1]+temp[0]
                new_lst[i+1] = temp[1]+new_lst[i+1]
        final_lst = [new_item for j, new_item in enumerate(new_lst) if j%2 == 0]
        return pd.Series(final_lst)
    cities_nfl[['nfl1', 'nfl2']] = pd.DataFrame(cities_nfl['NFL'].apply(split_and_keep))
    #----------------Preprocessing NFL data----------------------
    nfl_df_18 = nfl_df[nfl_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples:{nfl_df_18.shape[0]}") 
    # now keep only the columns we need
    cols_nfl = ['team', 'W', 'L']
    nfl_df_18 = nfl_df_18[cols_nfl]
    # now drop the rows containing information about division
    nfl_df_18 = nfl_df_18.replace('[AN]FC\s', np.nan, regex=True).dropna().reset_index(drop=True)
    # print(f"no of data samples after removing division row:{nfl_df_18.shape[0]}")
    # drop the asterisk(*) or plus(+) from team names
    nfl_df_18['team'] = nfl_df_18['team'].replace('[\*\+]', '', regex=True)
    # the W, L columns contain string, so convert them to integer
    nfl_df_18[['W', 'L']] = nfl_df_18[['W', 'L']].astype('int')
    # add a new row containing the intended ratio
    nfl_df_18['ratio'] = nfl_df_18['W'] / (nfl_df_18['W']+nfl_df_18['L'])
    # # remove space from city part of team names
    nfl_df_18['team'] = nfl_df_18['team'].replace({'New England':'NewEngland','New York':'NewYork','Kansas City':'KansasCity',
                                                   'Los Angeles': 'LosAngeles', 'Green Bay':'GreenBay', 'St. Louis':'StLouis',
                                                   'New Orleans':'NewOrleans','Tampa Bay':'TampaBay',  
                                                   'San Francisco':'SanFrancisco'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    nfl_df_18['team'] = nfl_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))

    #---------------------Merging---------------------------
    merged_nfl1 = cities_nfl.merge(nfl_df_18[['team', 'ratio']], how='left', left_on='nfl1', right_on='team')
    merged_nfl2 = merged_nfl1.merge(nfl_df_18[['team', 'ratio']], how='left', left_on='nfl2', right_on='team')
    # add the total win/loss ratio to the cities dataframe
    divide_value_nfl = 1*merged_nfl2['ratio_x'].notna() + 1*merged_nfl2['ratio_y'].notna() 
    cities_nfl['total_ratio'] = (merged_nfl2['ratio_x'] + merged_nfl2['ratio_y'].fillna(0))/divide_value_nfl
    
    
    #-----------------------------------------------------------------------------------
    #--------------------------------------MLB Data-------------------------------------
    #-----------------------------------------------------------------------------------
    # for this we need only MLB data. So, drop the cities that don't have any MLB teams
    cities_mlb = cities.dropna(subset=['MLB']) #drop cities w/o MLB teams
    cols_cities_mlb = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_mlb = cities_mlb.rename(columns=cols_cities_mlb)
    cities_mlb = cities_mlb[['metro', 'pop', 'MLB']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_mlb['pop'] = cities_mlb['pop'].astype('int')
    cities_mlb = cities_mlb.reset_index(drop=True) #reset_index

    cities_mlb[['mlb1', 'mlb2']] = pd.DataFrame(cities_mlb['MLB'].apply(split_and_keep))

    #--------------------Process MLB data----------------------
    mlb_df_18 = mlb_df[mlb_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples:{mlb_df_18.shape[0]}") 
    # now keep only the columns we need
    cols_mlb = ['team', 'W', 'L']
    mlb_df_18 = mlb_df_18[cols_mlb]
    # the W, L columns contain integer already. however make sure they are integer
    mlb_df_18[['W', 'L']] = mlb_df_18[['W', 'L']].astype('int')
    # add a new row containing the intended ratio
    mlb_df_18['ratio'] = mlb_df_18['W'] / (mlb_df_18['W']+mlb_df_18['L'])
    # remove space from city part of team names
    mlb_df_18['team'] = mlb_df_18['team'].replace({'Los Angeles': 'LosAngeles', 'New York':'NewYork', 'St. Louis':'StLouis', 
                                                   'Tampa Bay':'TampaBay', 'Kansas City':'KansasCity', 
                                                   'San Francisco':'SanFrancisco', 'San Diego':'SanDiego'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    mlb_df_18['team'] = mlb_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))

    #-------------Merge Data-----------------------
    merged_mlb1 = cities_mlb.merge(mlb_df_18[['team', 'ratio']], how='left', left_on='mlb1', right_on='team')
    merged_mlb2 = merged_mlb1.merge(mlb_df_18[['team', 'ratio']], how='left', left_on='mlb2', right_on='team')

    # add the total win/loss ratio to the cities dataframe
    divide_value_mlb = 1*merged_mlb2['ratio_x'].notna() + 1*merged_mlb2['ratio_y'].notna() 
    cities_mlb['total_ratio'] = (merged_mlb2['ratio_x'] + merged_mlb2['ratio_y'].fillna(0))/divide_value_mlb
    
    #-----------------------------------------------------------------------------------
    #--------------------------------------NBA Data-------------------------------------
    #-----------------------------------------------------------------------------------
    # for this we need only NBA data. So, drop the cities that don't have any NBA teams
    cities_nba = cities.dropna(subset=['NBA']) #drop cities w/o NBA teams
    cols_cities_nba = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_nba = cities_nba.rename(columns=cols_cities_nba)
    cities_nba = cities_nba[['metro', 'pop', 'NBA']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_nba['pop'] = cities_nba['pop'].astype('int')
    cities_nba = cities_nba.reset_index(drop=True) #reset_index

    cities_nba[['nba1', 'nba2']] = pd.DataFrame(cities_nba['NBA'].apply(split_and_keep))

    #--------------------Process NBA data------------------------
    nba_df_18 = nba_df[nba_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples:{nba_df_18.shape[0]}") 
    # now keep only the columns we need
    cols_nba = ['team', 'W', 'L']
    nba_df_18 = nba_df_18[cols_nba]
    # drop the asterisk(*) from team names
    nba_df_18['team'] = nba_df_18['team'].replace('\*\s\(.\)$|\s\(.*\)$', '', regex=True)
    # the W, L columns contain string. So convert those to integer
    nba_df_18[['W', 'L']] = nba_df_18[['W', 'L']].astype('int')
    # add a new row containing the intended ratio
    nba_df_18['ratio'] = nba_df_18['W'] / (nba_df_18['W']+nba_df_18['L'])
    # remove space from city part of team names
    nba_df_18['team'] = nba_df_18['team'].replace({'Los Angeles': 'LosAngeles', 'New York':'NewYork', 
                                                   'Golden State':'GoldenState', 'Oklahoma City':'OklahomaCity', 
                                                   'New Orleans':'NewOrleans', 'San Antonio':'SanAntonio'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    nba_df_18['team'] = nba_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))

    #-----------------Merge dataframe-------------------
    merged_nba1 = cities_nba.merge(nba_df_18[['team', 'ratio']], how='left', left_on='nba1', right_on='team')
    merged_nba2 = merged_nba1.merge(nba_df_18[['team', 'ratio']], how='left', left_on='nba2', right_on='team')

    # add the total win/loss ratio to the cities dataframe
    divide_value_nba = 1*merged_nba2['ratio_x'].notna() + 1*merged_nba2['ratio_y'].notna() 
    cities_nba['total_ratio'] = (merged_nba2['ratio_x'] + merged_nba2['ratio_y'].fillna(0))/divide_value_nba
    
    
    #-----------------------------------------------------------------------------------
    #--------------------------------------NHL Data-------------------------------------
    #-----------------------------------------------------------------------------------
    # for this we need only NHL data. So, drop the cities that don't have any NHL teams
    cities_nhl = cities.dropna(subset=['NHL']) #drop cities w/o NHL teams
    cols_cities_nhl = {'Metropolitan area':'metro', 'Population (2016 est.)[8]':'pop'} #rename columns for ease of use
    cities_nhl = cities_nhl.rename(columns=cols_cities_nhl)
    cities_nhl = cities_nhl[['metro', 'pop', 'NHL']] #keep only useful columns
    # population data was read as string. convert it into integer
    cities_nhl['pop'] = cities_nhl['pop'].astype('int')
    cities_nhl = cities_nhl.reset_index(drop=True) #reset_index

    cities_nhl[['nhl1', 'nhl2', 'nhl3']] = pd.DataFrame(cities_nhl['NHL'].apply(split_and_keep))

    #----------------------process NHL data----------------------------
    nhl_df_18 = nhl_df[nhl_df['year']==2018].dropna() #keep only 2018 data samples
    # print(f"no of data samples before dropping:{nhl_df_18.shape[0]}") #(3*8+1*7+4)
    # now keep only the columns we need
    cols_nhl = ['team', 'W', 'L', 'OL']
    nhl_df_18 = nhl_df_18[cols_nhl]
    # now drop the rows containing information about division
    nhl_df_18 = nhl_df_18.replace('\sDivision$', np.nan, regex=True).dropna().reset_index(drop=True)
    # drop the asterisk(*) from team names
    nhl_df_18['team'] = nhl_df_18['team'].replace('\*$', '', regex=True)
    # the W, L, OL columns contain string. So convert those to integer
    nhl_df_18[['W', 'L', 'OL']] = nhl_df_18[['W', 'L', 'OL']].astype('int')
    # add a new row containing the intended ratio
    #     nhl_df_18['ratio'] = nhl_df_18['W'] / (nhl_df_18['W']+nhl_df_18['L']+nhl_df_18['OL'])
    nhl_df_18['ratio'] = nhl_df_18['W'] / (nhl_df_18['L']+nhl_df_18['W'])
    # print(f"no of data samples before dropping:{nhl_df_18.shape[0]}") #(3*8+1*7)
    # remove space from city part of team names
    nhl_df_18['team'] = nhl_df_18['team'].replace({'Tampa Bay': 'TampaBay', 'New Jersey':'NewJersey', 
                                                   'Los Angeles': 'LosAngeles', 'St. Louis':'St.Louis', 
                                                   'San Jose':'SanJose', 'New York':'NewYork'}, regex=True)
    # remove city part from team names (even without the city part team names are unique)
    nhl_df_18['team'] = nhl_df_18['team'].apply(lambda name: ' '.join(name.split()[1:]))

    #------------------------- Merging---------------------------------
    merged_nhl1 = cities_nhl.merge(nhl_df_18[['team', 'ratio']], how='left', left_on='nhl1', right_on='team')
    merged_nhl2 = merged_nhl1.merge(nhl_df_18[['team', 'ratio']], how='left', left_on='nhl2', right_on='team')
    merged_nhl3 = merged_nhl2.merge(nhl_df_18[['team', 'ratio']], how='left', left_on='nhl3', right_on='team')

    # add the total win/loss ratio to the cities dataframe
    divide_value = 1*merged_nhl3['ratio_x'].notna() + 1*merged_nhl3['ratio_y'].notna() + 1*merged_nhl3['ratio'].notna()
    cities_nhl['total_ratio'] = (merged_nhl3['ratio_x'] + merged_nhl3['ratio_y'].fillna(0) 
                                 + merged_nhl3['ratio'].fillna(0))/divide_value
    
    #---------------------------------Specific for this task only---------------------------
    # till now we have done nothing but just copy pasting codes from previous tasks
    
    needed_cols = ['metro', 'total_ratio'] #columns we need for this task
    df_dict = {'NHL': cities_nhl[needed_cols], 'NBA': cities_nba[needed_cols],
               'MLB': cities_mlb[needed_cols], 'NFL': cities_nfl[needed_cols]} #keeping all dataframes in a dictionary  
                                                                               #for ease of access
    sports = ['NFL', 'NBA', 'NHL', 'MLB'] #name of sports
    p_values_dict = {'NFL':[], 'NBA':[], 'NHL':[], 'MLB':[]} #dictionary to store p values
    for i, clm1 in enumerate(sports):
        for j, clm2 in enumerate(sports):
            #merge the dataframes of two sports and the calculate their ttest score
            _df = pd.merge(df_dict[clm1], df_dict[clm2], 
                              how='inner', on='metro', suffixes=[f'_{clm1}_1', f'_{clm2}_2'])
            _pval = ttest_rel(_df[f"total_ratio_{clm1}_1"], _df[f"total_ratio_{clm2}_2"])[1]
            p_values_dict[clm1].append(_pval)
    #         print(temp_p)
    p_values = pd.DataFrame(p_values_dict, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
